In [18]:
from shelve import open as open_shelve
from matplotlib import pyplot as plt
%matplotlib qt
from kitt_net import FeedForwardNet
from kitt_learning import Backpropagation
from sklearn.metrics import confusion_matrix, accuracy_score
from numpy import newaxis, zeros

In [4]:
dataset = open_shelve('../examples/mnist/dataset_mnist_2K.ds')

In [3]:
how = {1: 'b.', -1: 'r.'}
for x, y in zip(dataset['x'], dataset['y']):
    plt.plot(x[0], x[1], how[y])
plt.grid()
plt.show()

In [15]:
net = FeedForwardNet(hidden=[20], tf_name='Sigmoid')

In [16]:
n_epoch = 10
learning_rate = 0.3
net.fit(x=dataset['x'], y=dataset['y'], x_val=dataset['x_val'], y_val=dataset['y_val'],
       learning_rate=learning_rate, n_epoch=n_epoch, batch_size=16)


--------------------------------------------------------------------
-- Network initialized.
	% problem dimension: 784
	% number of classes: 10
	% class labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
	% net structure: [784, 20, 10]
	% net transfer function: Sigmoid

--------------------------------------------------------------------
-- Learning has started...
	% problem dimension: 784
	% number of training samples: 20000
	% number of validation samples: 10000
	% learning rate: 0.3
	% mini-batch size: 16
	% maximum number of epochs (t.c.): 10
	% maximum number of stable epochs (t.c.): inf
	% required accuracy (t.c.): inf
	% required error (t.c.): -inf


epoch   on training data      on validation data       epoch time          
-------------------------------------------------------------------
 1	  0.87/0.0102		0.88/0.0097		7.0210 s
 2	  0.90/0.0081		0.90/0.0082		6.6529 s
 3	  0.91/0.0072		0.90/0.0075		6.2872 s
 4	  0.92/0.0062		0.91/0.0069		6.6656 s
 5	  0.93/0.0056		0.92/0.0065		6.5103 s
 

In [19]:
net_obd = net.copy_()
net_obd.learning = Backpropagation({'self': None})
net_mag = net.copy_()
net_mag.learning = Backpropagation({'self': None})
net_karnin = net.copy_()
net_karnin.learning = Backpropagation({'self': None})
net_random = net.copy_()
net_random.learning = Backpropagation({'self': None})

In [5]:
print net.w[0]
print '---------------'
print 'MAGNITUDE'
print abs(net.w)[0]

print 'KITT'
print abs(net.w-net.w_init)[0]

print 'KARNIN'
changes_ = [zeros(shape=w_i.shape) for w_i in net.w]
for ep_i in range(net.dw_i):
    for l_i in range(len(changes_)):
        den = (net.learning.kw['learning_rate']*(net.w[l_i]-net.w_init[l_i]))
        changes_[l_i] += net.dw_container[l_i][ep_i]**2*(net.w[l_i]/den)
print changes_[0]
        
print 'OBD'                    
print net.saliency[0]            

[[  0.57064674  61.62288871]
 [-51.26244559  -0.        ]]
---------------
MAGNITUDE
[[  0.57064674  61.62288871]
 [ 51.26244559   0.        ]]
KITT
[[  0.56044682  61.59216834]
 [ 49.97506773   1.55741384]]
KARNIN
[[ -5823.74815686  12434.54425503]
 [  4439.84284332      0.        ]]
OBD
[[ 5.9384484   9.61292656]
 [ 0.16123191  6.86683045]]


In [ ]:
net.prune(req_acc=0.93, n_epoch=5, measure='random', retrain=False)


--------------------------------------------------------------------
-- Pruning has started...
	% net initial structure: [784, 20, 10]
	% net initial number of synapses (w, b): (15880, 30)
	% min required accuracy (s.c.): 0.93
	% max required error (s.c.): 0.0
	% maximum number of re-training epochs (gu.c.): 5
	% number of stable iterations (gu.c.): 10


step    trying to cut     structure           left synapses (w/b)    retrained      next level     step time      
--------------------------------------------------------------------------------------------------------------
0       0                 [784, 20, 10]       (15880, 30)            None           75             None  
1       11910             [780, 20, 10]       (3970, 30)             yes            75             18.1261 s
2       2977              [564, 11, 10]       (589, 21)              no             50             13.3203 s
3       1985              [726, 19, 10]       (1889, 29)             no             35      

In [21]:
labels = [label for label in net.labels if label in dataset['y_val']]
y_pred = [net.predict(x)[0][0] for x, y in net.v_data]
print 'Acc:', accuracy_score(y_true=dataset['y_val'], y_pred=y_pred)
cm = confusion_matrix(y_true=dataset['y_val'], y_pred=y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, newaxis]
plt.imshow(cm, aspect='auto', interpolation='none', vmin=0, vmax=1)
plt.xticks(range(len(labels)), labels)
plt.yticks(range(len(labels)), labels)
plt.grid()
plt.colorbar()
plt.show()
print cm
for i in range(cm.shape[0]):
    print i, net.labels[i], sorted([(net.labels[j], round(cm[i, j], 2)) for j in range(cm.shape[1])], key=lambda x:x[1], reverse=True)[:5]


Acc: 0.9267
[[ 0.96770938  0.          0.00605449  0.00201816  0.00100908  0.00706357
   0.00504541  0.00100908  0.00807265  0.00201816]
 [ 0.          0.96804511  0.00281955  0.0075188   0.00281955  0.0037594
   0.00281955  0.0037594   0.00845865  0.        ]
 [ 0.00606061  0.0030303   0.9040404   0.01414141  0.01010101  0.00505051
   0.01515152  0.01616162  0.02525253  0.0010101 ]
 [ 0.0038835   0.00097087  0.01165049  0.87378641  0.00194175  0.05242718
   0.0038835   0.00873786  0.03495146  0.00776699]
 [ 0.00101729  0.00610376  0.00305188  0.          0.95930824  0.
   0.00203459  0.00610376  0.00508647  0.017294  ]
 [ 0.01420765  0.00218579  0.00983607  0.0284153   0.00765027  0.8852459
   0.01967213  0.00874317  0.01967213  0.00437158]
 [ 0.00517063  0.00310238  0.00620476  0.          0.00827301  0.01344364
   0.95966908  0.          0.0041365   0.        ]
 [ 0.00366972  0.00550459  0.01376147  0.00458716  0.00917431  0.00366972
   0.          0.94587156  0.00183486  0.01192661